In [7]:
import pandas as pd
import keras
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras.preprocessing.text import one_hot
from keras.layers import merge,Reshape, Flatten
from keras import backend as K
from sklearn.preprocessing import LabelEncoder

import numpy as np

data=pd.read_csv('../data/regular_season_results.csv')
data['wloc']=data['wloc'].map({'H':1,'A':0})
print(data.tail())
teams = pd.read_csv('../data/teams.csv')
nteams=len(teams['id'])
win=pd.get_dummies(data['wteam'])
lose=pd.get_dummies(data['lteam'])
input_data=np.array(win + lose)

input_1 = Input(batch_shape=(None,1),dtype='float32',name='w_team')
input_2 = Input(batch_shape=(None,1),dtype='float32',name='l_team')

emb1 = Embedding(output_dim=64,input_dim=857,input_length=1)(input_1)
emb2 = Embedding(output_dim=64,input_dim=857,input_length=1)(input_2)
emba = Flatten()(emb1)
embb = Flatten()(emb2)

game = merge([emb1,emb2],mode='concat',concat_axis=1)
flatten = Reshape((128,))(game)
game = Dense(64, activation='relu')(flatten)

#needs to be a custom layer??
perf1 = merge([emb1,game], mode = lambda x: K.sum(x[0]*x[1]),output_shape=(1,))
perf2 = merge([emb2,game], mode = lambda x: K.sum(x[0]*x[1]),output_shape=(1,))
perf = merge([perf1,perg2],mode= lambda x: x[0]-x[1],output_shape)
pred=Activation('sigmoid')(perf1 -perf2)

#flatten = Reshape((128,))(game)
#game = Dense(64, activation='relu')(flatten)
#pred = Dense(1,activation='sigmoid')(game)
model=Model(input=(input_1,input_2),output=pred)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.fit([np.array(data['wteam']),np.array(data['lteam'])],np.array([1 for i in range(0,len(input_data))]))

      season  daynum  wteam  wscore  lteam  lscore  wloc  numot
91219      S     132    592      61    640      60   NaN    0.0
91220      S     132    671      69    670      55   NaN    0.0
91221      S     132    779      65    825      61   NaN    0.0
91222      S     132    811      82    605      81   NaN    1.0
91223      S     132    830      72    577      63   NaN    0.0


Exception: Output tensors to a Model must be Keras tensors. Found: Tensor("Sigmoid_1:0", shape=(), dtype=float32)

In [2]:
from keras.layers.core import Flatten
input_data=np.array(win + lose)

#Implementation of Terry-Bradley ANN

model=Sequential()
model.add(Dense(1,input_dim=nteams))
model.add(Activation('sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(input_data,[1 for i in range(0,len(input_data))])


Epoch 1/10
91224/91224 [==============================] - 1s - loss: 0.2628 - acc: 0.9858     
Epoch 2/10
91224/91224 [==============================] - 1s - loss: 0.0179 - acc: 1.0000     
Epoch 3/10
91224/91224 [==============================] - 1s - loss: 0.0011 - acc: 1.0000     
Epoch 4/10
91224/91224 [==============================] - 1s - loss: 6.2976e-05 - acc: 1.0000     
Epoch 5/10
91224/91224 [==============================] - 1s - loss: 3.7649e-06 - acc: 1.0000     
Epoch 6/10
91224/91224 [==============================] - 1s - loss: 2.3159e-07 - acc: 1.0000     
Epoch 7/10
91224/91224 [==============================] - 1s - loss: 1.2004e-07 - acc: 1.0000     
Epoch 8/10
91224/91224 [==============================] - 1s - loss: 1.1949e-07 - acc: 1.0000     
Epoch 9/10
91224/91224 [==============================] - 1s - loss: 1.1938e-07 - acc: 1.0000     
Epoch 10/10
91224/91224 [==============================] - 1s - loss: 1.1933e-07 - acc: 1.0000     


In [3]:
# rankings looks reasonable 

weights=model.layers[0].get_weights()
    
print(weights[0].shape)
ranks=zip(teams['name'],map(lambda x:x[0],weights[0]))
ranks=sorted(ranks,key=lambda x: x[1],reverse=True)
print(ranks)

(356, 1)
[('Duke', 1.5817878), ('Kansas', 1.5185366), ('Kentucky', 1.2558224), ('North Carolina', 1.1737632), ('Syracuse', 1.1448064), ('Connecticut', 1.1141794), ('Arizona', 1.0802886), ('Michigan St', 1.0385598), ('Florida', 1.0196065), ('Gonzaga', 1.0105392), ('Texas', 0.9768309), ('Louisville', 0.97504449), ('Cincinnati', 0.97021908), ('Memphis', 0.95779139), ('Wisconsin', 0.9278518), ('Illinois', 0.90887874), ('Pittsburgh', 0.90706897), ('Maryland', 0.90286481), ('Oklahoma', 0.8937549), ('UCLA', 0.8909415), ('Ohio St', 0.88501954), ('Oklahoma St', 0.87141299), ('Stanford', 0.86533934), ('Villanova', 0.86345059), ('Georgetown', 0.84876126), ('Xavier', 0.83903414), ('Marquette', 0.81679571), ('Tennessee', 0.81606609), ('Missouri', 0.76348424), ('New Mexico', 0.73766464), ('Notre Dame', 0.71736729), ('Wake Forest', 0.71504498), ('Indiana', 0.71109766), ('Purdue', 0.70197904), ('Creighton', 0.69647616), ('California', 0.66604292), ('NC State', 0.66436505), ('UNLV', 0.66119802), ('Butl